# Evaluation vertical Prototype

In [15]:
import define
import json
from os import listdir
import csv
import pandas as pd
from elasticsearch import Elasticsearch
es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [16]:
# make dictionary with all topics
import xml.etree.ElementTree as ET
tree = ET.parse('topics.xml')
# tree = ET.parse('topics_selected.xml')
topics = tree.findall('topic')

topicsDic = {}

for topic in topics:
    title = topic.find('title').text
    number = topic.find('number').text
    topicsDic[number] = title

topicsDic

{'1': 'Should teachers get tenure?',
 '2': 'Is vaping with e-cigarettes safe?',
 '3': 'Should insider trading be allowed?',
 '4': 'Should corporal punishment be used in schools?',
 '5': 'Should social security be privatized?',
 '6': 'Is a college education worth it?',
 '7': 'Should felons who have completed their sentence be allowed to vote?',
 '8': 'Should abortion be legal?',
 '9': 'Should students have to wear school uniforms?',
 '10': 'Should any vaccines be required for children?',
 '11': 'Should performance-enhancing drugs be accepted in sports?',
 '12': 'Should birth control pills be available over the counter?',
 '13': 'Can alternative energy effectively replace fossil fuels?',
 '14': 'Is sexual orientation determined at birth?',
 '15': 'Should animals be used for scientific or commercial testing?',
 '16': 'Should prescription drugs be advertised directly to consumers?',
 '17': 'Should recreational marijuana be legal?',
 '18': 'Should churches remain tax-exempt?',
 '19': 'Shoul

## Define the search for the vertical prototype

In [17]:
def search_vertical_prototype_OCR(query, num_results):
    # make query
    body = {
        "from":0,
        "size":num_results,
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":query}},
                    {"match": {"ocr_text":{"query":query, "boost":5}}}  # ocr_text sollte einen natürlichen boost besitzen, da die texte viel kürzer sind. Vielleicht muss dieser auch abgeschwächt werden?
                ]
            }   
        }
    }
    res = es.search(index="boromir_index", body=body)
    # get ID's from retrieved documents
    result_ids_scores = []
    results = res.get('hits').get('hits')

    for doc in results:
        id = doc.get('_id')
        score = doc.get('_score')
        result_ids_scores.append([id, score])

    return result_ids_scores

## Make evaluation for query expansion

In [18]:
all_ranks_pro = []
all_ranks_con = []
numbers = []
true_pos_pro = []
true_pos_con = []
avg_rank_pro = []
avg_rank_con = []

result_ids_pro = []
result_ids_con = []

removed_duplicates_pro = 0
removed_duplicates_con = 0

noTPpro = 0
noTPcon = 0

duplicates = []

num_results = 10 # number of result id's we want to get

for topic in topicsDic:
    query = topicsDic.get(topic)
    query_pro = query + " pro"
    query_con = query + " anti"
    dup_result_ids_scores_pro = search_vertical_prototype_OCR(query_pro, num_results*100)
    dup_result_ids_scores_con = search_vertical_prototype_OCR(query_con, num_results*100)

    result_ids_pro = []
    result_ids_con = []

    # make sure any id is only present in either pro or con:
    for id, score in dup_result_ids_scores_pro:
        include = True
        for _id, _score in dup_result_ids_scores_con:
            if id == _id and score <= _score:
                include = False
                removed_duplicates_con += 1
                break
        if include:
            result_ids_pro.append(id)
        if len(result_ids_pro) == num_results: break
            
    for id, score in dup_result_ids_scores_con:
        include = True
        for _id, _score in dup_result_ids_scores_pro:
            if id == _id and score <= _score:
                include = False
                removed_duplicates_pro += 1
                break
        if include:
            result_ids_con.append(id)
        if len(result_ids_con) == num_results: break
    
    for id in result_ids_pro:
        if id in result_ids_con:
            print(id)
    
    duplicates.append([topic, removed_duplicates_pro, removed_duplicates_con])
    removed_duplicates_pro = 0
    removed_duplicates_con = 0
    no_TPpro = 0
    no_TPcon = 0
    # print(result_ids_pro)
    # print(result_ids_con)
    # print(len(result_ids_pro))
    # print(len(result_ids_con))

        

    print(query, "\n======================================")

    TP_pro = 0
    TP_con = 0
    total_rank_pro = 0 # the smaller the better
    total_rank_con = 0 # the smaller the better

    # check each result id if it fits the topic (pro)
    for id in result_ids_pro:

        path = define.imagePath() + "/images/"  + id[0:3] + "/" + id + "/pages/"
        try:
            file = listdir(path)
            path = path +  file[0] + "/rankings.jsonl"

            lines = []
            for line in open(path, 'r'):
                lines.append(json.loads(line))

            print(lines[0]['query'])

            if lines[0]['topic'] == topic:
                TP_pro += 1
                rank = lines[0]['rank']
                total_rank_pro += rank
        except: print("not found!")

    # check each result id if it fits the topic (con)
    for id in result_ids_con:

        path = define.imagePath() + "/images/"  + id[0:3] + "/" + id + "/pages/"
        try:
            file = listdir(path)
            path = path +  file[0] + "/rankings.jsonl"

            lines = []
            for line in open(path, 'r'):
                lines.append(json.loads(line))

            print(lines[0]['query'])

            if lines[0]['topic'] == topic:
                TP_con += 1
                rank = lines[0]['rank']
                total_rank_con += rank
        except: print("not found2")
    
    # print search performance for this topic
        
    print("Number of pictures that fit to the query (Pro): ", TP_pro, "/", num_results)
    print("Precision (Pro): ", TP_pro/num_results)
    print("Total Rank for Query (Pro): ", total_rank_pro)
    print("....................................")
    print("Number of pictures that fit to the query (Con): ", TP_con, "/", num_results)
    print("Precision (Con): ", TP_con/num_results)
    print("Total Rank for Query (Con): ", total_rank_con, "\n")

    all_ranks_pro.append(total_rank_pro)
    all_ranks_con.append(total_rank_con)
    numbers.append(topic)
    true_pos_pro.append(TP_pro)
    true_pos_con.append(TP_con)
    if(TP_pro != 0):
        avg_rank_pro.append(round(total_rank_pro/TP_pro, 2))
    else:
        avg_rank_pro.append(0)
        noTPpro += 1
    if(TP_con != 0):
        avg_rank_con.append(round(total_rank_con/TP_con, 2))
    else:
        avg_rank_con.append(0)
        noTPcon += 1
    
#print(removed_duplicates)
# make final dataframe
df = pd.DataFrame({'Topic': numbers, 'Rank Pro' : all_ranks_pro,  'TP Pro' : true_pos_pro, 'AVG_Rank Pro' : avg_rank_pro, 'Rank Con' : all_ranks_con, 'TP Con' : true_pos_con, 'AVG_Rank Con' : avg_rank_con})
df.to_csv('evaluation_results/0+OCRevaluation_topic.tsv', index = False, sep='\t', line_terminator = '\r\n')

C:\Users\thilo\AppData\Local\Temp/ipykernel_3948/1553798195.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="boromir_index", body=body)
C:\Users\thilo\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Should teachers get tenure? 
marijuana recreational use quote
death penalty quote
Should the penny stay in circulation?
uniforms are a symbol of school discipline
Should tablets replace textbooks in schools?
teacher tenure quote
teacher tenure quote
abortion stats
Should abortion be legal?
effects of using tablets instead of textbooks in education
election day holiday anti
teacher tenure anti
medical marijuana anti
compulsory vaccination for children meme
obesity disease anti
medical marijuana anti
election day holiday meme
should teachers get tenure?
net neutrality anti
eliminate penny anti
Number of pictures that fit to the query (Pro):  2 / 10
Precision (Pro):  0.2
Total Rank for Query (Pro):  115
....................................
Number of pictures that fit to the query (Con):  2 / 10
Precision (Con):  0.2
Total Rank for Query (Con):  62 

Is vaping with e-cigarettes safe? 
abortion stats
abortion stats
abortion protests
if you're not ready to quit, you may want to choose e-ciga

#### Anzahl von pro oder con Seiten, für die KEINE true positives gefunden werden konnten

In [19]:
print(noTPpro)
print(noTPcon)

3
5


#### Durchschnittliche Anzahl verworfener Bilder auf pro ODER con Seite

In [20]:
import statistics

In [21]:
#erst pro dann con duplicates
#print(duplicates)

pro = []
con = []
for i in duplicates:
    #print(i[1])
    #print(i[2])
    #print("new")
    pro.append(i[1])
    con.append(i[2])

average_pro = statistics.mean(pro)
average_con = statistics.mean(con)

print(average_pro)
print(average_con)

87.87755102040816
67.93877551020408


In [14]:
#duplicates